# 要約 
このJupyter Notebookは、Kaggleの「LMSYS - Chatbot Arena 人間による好み予測チャレンジ」において、ユーザーが2つのチャットボットの応答からどちらを好むかを予測する問題に取り組んでいます。具体的には、与えられたプロンプトに対する2つの異なるモデル（モデルAおよびモデルB）の応答を評価し、どのモデルが選好されるか、または引き分けになるかを予測するための機械学習モデルを構築しています。

### 主要な手法とライブラリ:
1. **ライブラリの使用**:
   - **Pandas**: データセットの読み込みや処理に使用。
   - **NumPy**: 数値計算をサポート。
   - **TensorFlow**: 深層学習モデルの構築とトレーニングに使用。
   - **Transformers**: 自然言語処理のためのBERTモデルを利用するライブラリ。

2. **データ処理**:
   - データセットに含まれるプロンプトと2つの応答から特徴量を抽出し、モデルのトレーニングに必要な形式に変換します。
   - 特徴量として、プロンプト、モデルAおよびモデルBの応答、そして勝者を示すラベルを使用します。

3. **モデル構築**:
   - BERTトークナイザーを用いて、テキストデータをトークン化します。
   - BERTモデル自体を使用して、プロンプトおよび応答を埋め込み（embedding）として変換します。
   - これらの埋め込みを結合し、その後に全結合層（Dense Layer）を通じて最終的な予測を行うモデルを構築します。

4. **トレーニングと評価**:
   - トレーニングデータとバリデーションデータに分割し、モデルのトレーニングを実施します。
   - 最後に、テストデータに対して予測を行い、その結果をCSVファイルとして出力します。

このNotebookは、自然言語処理における先進的な技術（BERTなど）を用いた深層学習の適用例を示しており、ユーザーの嗜好を具体的な数値に落とし込むためのプロセスを包括的に扱っています。最終的な出力は、各応答の勝者モデル（モデルA、モデルB、引き分け）の確率を示したCSVファイル「submission.csv」として保存されます。

---


# 用語概説 
以下に、初心者がつまずきそうな専門用語について簡単な解説を示します。このリストは、ノートブックの内容に特有の用語や、実務経験がないと馴染みが薄い可能性のあるものに焦点を当てています。

1. **TPU（Tensor Processing Unit）**:
   - Googleによって開発された専用のハードウェアアクセラレーターで、ディープラーニングモデルのトレーニングや推論を高速化するために使用されます。GPUよりも特定のタスクに特化しているため、高い性能が期待できます。

2. **トークナイザー（Tokenizer）**:
   - テキストをモデルに入力するための形式（トークン）に変換するツールです。トークナイザーは、単語や文字を特定の式にマッピングし、モデルが処理できるようにします。例えば、BERTの場合は、語彙に基づいてテキストをトークンに分割します。

3. **アテンションマスク（Attention Mask）**:
   - モデルがどのトークンに注目するかを制御するためのマスクです。特にパディングトークン（無意味なトークン）を無視するために使われ、効率的にトレーニングや推論を行うための重要な要素です。

4. **埋め込み（Embedding）**:
   - 単語や文のような離散的な情報を連続的なベクトルに変換する技術です。これにより、機械学習モデルは言語の意味的な関係をより良く理解することができます。BERTモデルで得られる埋め込みは、文脈に応じた特徴量を持ちます。

5. **CLSトークン（[CLS] token）**:
   - BERTモデルで特別に使用されるトークンで、文全体の情報を要約するために使用されます。BERTの出力の最初の部分に位置し、このトークンの埋め込みを使って分類タスクの結果を予測することが一般的です。

6. **ドロップアウト（Dropout）**:
   - モデルの過学習を防ぐための手法で、トレーニング中に一定の確率で神経元（ユニット）を無効にすることにより、モデルの汎化性能を向上させる技術です。ドロップアウト率は、無効にするユニットの割合を示します。

7. **カテゴリカルラベル（Categorical Label）**:
   - 複数のクラスに分類される出力に対して、その各クラスを1-hotエンコーディングの形式で表現したものです。この形式により、モデルがクラス間の関係を学習しやすくなります。

8. **ステートフル（Stateful）**:
   - モデルがトレーニング中に過去の状態を保持する特性を意味します。これにより、長いシーケンスデータの処理が可能になり、特にリカレントニューラルネットワーク（RNN）に関連しています。

9. **バッチ（Batch）**:
   - モデルが一度に処理するデータのサブセットを指します。大規模データセットのトレーニングでは、全データを一度に処理するのではなく、バッチ単位で処理することでメモリを効率的に使用し、モデルのパラメータを更新します。

10. **エポック（Epoch）**:
    - トレーニングプロセスにおいて、全トレーニングデータセットを1回通すことを指します。複数エポックでトレーニングすることにより、モデルのパラメータが更新され、性能が向上することが期待されます。

これらの用語は、ノートブック内で広く使用されているため、理解しておくとタスクの実行やデバッグに役立つでしょう。

---


<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
```

</div>
<div class="column-right">

# 日本語訳

```python
# このPython 3環境には、多くの便利な解析ライブラリがインストールされています
# これはkaggle/pythonのDockerイメージによって定義されています: https://github.com/kaggle/docker-python
# 例として、いくつかの便利なパッケージを読み込むことができます

import numpy as np # 線形代数のためのライブラリ
import pandas as pd # データ処理のためのライブラリ、CSVファイルの入出力 (例: pd.read_csv)

# 入力データファイルは、読み取り専用の"../input/"ディレクトリにあります
# 例えば、これを実行すると (クリックして実行したりShift+Enterを押すことで) 入力ディレクトリ内のすべてのファイルをリストします

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename)) # 各ファイルのパスを表示します

# 現在のディレクトリ (/kaggle/working/) に最大20GBまで書き込むことができます
# このディレクトリは「Save & Run All」を使用してバージョンを作成した際に出力として保存されます
# 一時ファイルは/kaggle/temp/に書き込むこともできますが、現在のセッションの外では保存されません
```

</div>
</details>

In [ ]:
# このPython 3環境には、多くの便利な解析ライブラリがインストールされています
# これはkaggle/pythonのDockerイメージによって定義されています: https://github.com/kaggle/docker-python
# 例として、いくつかの便利なパッケージを読み込むことができます

import numpy as np # 線形代数のためのライブラリ
import pandas as pd # データ処理のためのライブラリ、CSVファイルの入出力 (例: pd.read_csv)

# 入力データファイルは、読み取り専用の"../input/"ディレクトリにあります
# 例えば、これを実行すると (クリックして実行したりShift+Enterを押すことで) 入力ディレクトリ内のすべてのファイルをリストします

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename)) # 各ファイルのパスを表示します

# 現在のディレクトリ (/kaggle/working/) に最大20GBまで書き込むことができます
# このディレクトリは「Save & Run All」を使用してバージョンを作成した際に出力として保存されます
# 一時ファイルは/kaggle/temp/に書き込むこともできますが、現在のセッションの外では保存されません

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
train=pd.read_csv('/kaggle/input/lmsys-chatbot-arena/train.csv')
test=pd.read_csv('/kaggle/input/lmsys-chatbot-arena/test.csv')
```

</div>
<div class="column-right">

# 日本語訳

```python
train=pd.read_csv('/kaggle/input/lmsys-chatbot-arena/train.csv') # トレーニングデータをCSVファイルから読み込みます
test=pd.read_csv('/kaggle/input/lmsys-chatbot-arena/test.csv') # テストデータをCSVファイルから読み込みます
```

</div>
</details>

In [ ]:
train=pd.read_csv('/kaggle/input/lmsys-chatbot-arena/train.csv') # トレーニングデータをCSVファイルから読み込みます
test=pd.read_csv('/kaggle/input/lmsys-chatbot-arena/test.csv') # テストデータをCSVファイルから読み込みます

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
import tensorflow as tf
# detect and init the TPU


```

</div>
<div class="column-right">

# 日本語訳

```python
import tensorflow as tf # TensorFlowライブラリをインポートします
# TPUを検出して初期化します
# TPU（Tensor Processing Unit）は、特にディープラーニングのトレーニングと推論に最適化されたハードウェアアクセラレーターです
```

</div>
</details>

In [ ]:
import tensorflow as tf # TensorFlowライブラリをインポートします
# TPUを検出して初期化します
# TPU（Tensor Processing Unit）は、特にディープラーニングのトレーニングと推論に最適化されたハードウェアアクセラレーターです

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

## Import libraries

</div>
<div class="column-right">

# 日本語訳

## ライブラリのインポート

</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
import pandas as pd
import tensorflow as tf
import numpy as np
import transformers 
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer

```

</div>
<div class="column-right">

# 日本語訳

```python
import pandas as pd # データ処理のためのライブラリをインポートします
import tensorflow as tf # TensorFlowライブラリをインポートします
import numpy as np # 数値計算のためのライブラリをインポートします
import transformers # Transformersライブラリをインポートします。自然言語処理で使われるモデルのためのライブラリです
from sklearn.model_selection import train_test_split # データをトレーニングセットとテストセットに分割するための関数をインポートします
from transformers import BertTokenizer # BERTモデル用のトークナイザーをインポートします。テキストをトークンに変換する役割があります
```

</div>
</details>

In [ ]:
import pandas as pd # データ処理のためのライブラリをインポートします
import tensorflow as tf # TensorFlowライブラリをインポートします
import numpy as np # 数値計算のためのライブラリをインポートします
import transformers # Transformersライブラリをインポートします。自然言語処理で使われるモデルのためのライブラリです
from sklearn.model_selection import train_test_split # データをトレーニングセットとテストセットに分割するための関数をインポートします
from transformers import BertTokenizer # BERTモデル用のトークナイザーをインポートします。テキストをトークンに変換する役割があります

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
tokenizer = BertTokenizer.from_pretrained('/kaggle/input/bert-base-uncased')
```

</div>
<div class="column-right">

# 日本語訳

```python
tokenizer = BertTokenizer.from_pretrained('/kaggle/input/bert-base-uncased') # 事前学習済みのBERTトークナイザーを指定されたパスからロードします
# このトークナイザーは、テキストをBERTモデルの入力形式に変換するために使用されます
# 'bert-base-uncased'は小文字のみのBERTモデルで、事前学習済みの重みを含んでいます
```

</div>
</details>

In [ ]:
tokenizer = BertTokenizer.from_pretrained('/kaggle/input/bert-base-uncased') # 事前学習済みのBERTトークナイザーを指定されたパスからロードします
# このトークナイザーは、テキストをBERTモデルの入力形式に変換するために使用されます
# 'bert-base-uncased'は小文字のみのBERTモデルで、事前学習済みの重みを含んでいます

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
from transformers import TFBertModel

# Load BERT model
bert_model = TFBertModel.from_pretrained('/kaggle/input/bert-base-uncased')

```

</div>
<div class="column-right">

# 日本語訳

```python
from transformers import TFBertModel # TensorFlow用のBERTモデルをインポートします

# BERTモデルをロードします
bert_model = TFBertModel.from_pretrained('/kaggle/input/bert-base-uncased') # 指定されたパスから事前学習済みのBERTモデルをロードします
# このモデルは、自然言語処理タスクにおいて特徴量抽出や埋め込みを生成するために使用されます
```

</div>
</details>

In [ ]:
from transformers import TFBertModel # TensorFlow用のBERTモデルをインポートします

# BERTモデルをロードします
bert_model = TFBertModel.from_pretrained('/kaggle/input/bert-base-uncased') # 指定されたパスから事前学習済みのBERTモデルをロードします
# このモデルは、自然言語処理タスクにおいて特徴量抽出や埋め込みを生成するために使用されます

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
for i in train.index:
    if train.loc[i,'winner_model_a']==1:
        train.loc[i,'winner']=0
    elif train.loc[i,'winner_model_b']==1:
        train.loc[i,'winner']=1
    else :
        train.loc[i,'winner']=2
```

</div>
<div class="column-right">

# 日本語訳

```python
for i in train.index: # トレーニングデータのすべてのインデックスに対してループします
    if train.loc[i,'winner_model_a'] == 1: # モデルAが勝者である場合
        train.loc[i,'winner'] = 0 # 勝者の列に0を設定します（モデルAが勝った場合）
    elif train.loc[i,'winner_model_b'] == 1: # モデルBが勝者である場合
        train.loc[i,'winner'] = 1 # 勝者の列に1を設定します（モデルBが勝った場合）
    else:
        train.loc[i,'winner'] = 2 # いずれのモデルも勝者でない場合は、勝者の列に2を設定します（引き分けと見なす）
```

</div>
</details>

In [ ]:
for i in train.index: # トレーニングデータのすべてのインデックスに対してループします
    if train.loc[i,'winner_model_a'] == 1: # モデルAが勝者である場合
        train.loc[i,'winner'] = 0 # 勝者の列に0を設定します（モデルAが勝った場合）
    elif train.loc[i,'winner_model_b'] == 1: # モデルBが勝者である場合
        train.loc[i,'winner'] = 1 # 勝者の列に1を設定します（モデルBが勝った場合）
    else:
        train.loc[i,'winner'] = 2 # いずれのモデルも勝者でない場合は、勝者の列に2を設定します（引き分けと見なす）

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
features=['prompt','response_a','response_b','winner']
```

</div>
<div class="column-right">

# 日本語訳

```python
features = ['prompt', 'response_a', 'response_b', 'winner'] # 使用する特徴量のリストを定義します
# 'prompt'はユーザーからの入力文、'response_a'はモデルAの応答、'response_b'はモデルBの応答、'winner'は勝者を示します
```

</div>
</details>

In [ ]:
features = ['prompt', 'response_a', 'response_b', 'winner'] # 使用する特徴量のリストを定義します
# 'prompt'はユーザーからの入力文、'response_a'はモデルAの応答、'response_b'はモデルBの応答、'winner'は勝者を示します

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
train_data=train[features]
```

</div>
<div class="column-right">

# 日本語訳

```python
train_data = train[features] # トレーニングデータから指定した特徴量のみを抽出して、新しいデータフレームtrain_dataを作成します
# train_dataには'prompt', 'response_a', 'response_b', 'winner'の情報が含まれます
```

</div>
</details>

In [ ]:
train_data = train[features] # トレーニングデータから指定した特徴量のみを抽出して、新しいデータフレームtrain_dataを作成します
# train_dataには'prompt', 'response_a', 'response_b', 'winner'の情報が含まれます

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
X_train,X_val=train_test_split(train_data,test_size=0.2,random_state=42)

```

</div>
<div class="column-right">

# 日本語訳

```python
X_train, X_val = train_test_split(train_data, test_size=0.2, random_state=42) # トレーニングデータをトレーニングセットとバリデーションセットに分割します
# test_size=0.2は全体の20%をバリデーションセットとし、random_state=42は再現性のための乱数シードを設定します
# X_trainにはトレーニングデータが、X_valにはバリデーションデータが含まれます
```

</div>
</details>

In [ ]:
X_train, X_val = train_test_split(train_data, test_size=0.2, random_state=42) # トレーニングデータをトレーニングセットとバリデーションセットに分割します
# test_size=0.2は全体の20%をバリデーションセットとし、random_state=42は再現性のための乱数シードを設定します
# X_trainにはトレーニングデータが、X_valにはバリデーションデータが含まれます

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
# Tokenize function
def tokenize_function(df):
    prompt_encodings = tokenizer(
        df['prompt'].tolist(),
        padding='max_length',
        truncation=True,
        max_length=128,
        return_tensors='tf'
    )
    response_a_encodings = tokenizer(
        df['response_a'].tolist(),
        padding='max_length',
        truncation=True,
        max_length=128,
        return_tensors='tf'
    )
    response_b_encodings = tokenizer(
        df['response_b'].tolist(),
        padding='max_length',
        truncation=True,
        max_length=128,
        return_tensors='tf'
    )
    return prompt_encodings, response_a_encodings, response_b_encodings 


train_prompt_encodings, train_response_a_encodings, train_response_b_encodings = tokenize_function(X_train)
val_prompt_encodings, val_response_a_encodings, val_response_b_encodings = tokenize_function(X_val)

```

</div>
<div class="column-right">

# 日本語訳

```python
# トークナイズ関数
def tokenize_function(df): # 引数としてデータフレームを受け取るトークナイズ関数を定義します
    prompt_encodings = tokenizer( # プロンプトのトークン化
        df['prompt'].tolist(), # 'prompt'列の内容をリストに変換します
        padding='max_length', # 最大長までパディングを行います
        truncation=True, # 長すぎる入力を切り捨てます
        max_length=128, # 最大長を128に設定します
        return_tensors='tf' # TensorFlowテンソル形式で返します
    )
    response_a_encodings = tokenizer( # モデルAの応答のトークン化
        df['response_a'].tolist(), 
        padding='max_length',
        truncation=True,
        max_length=128,
        return_tensors='tf'
    )
    response_b_encodings = tokenizer( # モデルBの応答のトークン化
        df['response_b'].tolist(),
        padding='max_length',
        truncation=True,
        max_length=128,
        return_tensors='tf'
    )
    return prompt_encodings, response_a_encodings, response_b_encodings # トークン化されたプロンプト、応答A、応答Bを返します

# トークナイズを実行し、トレーニングデータとバリデーションデータを処理します
train_prompt_encodings, train_response_a_encodings, train_response_b_encodings = tokenize_function(X_train)
val_prompt_encodings, val_response_a_encodings, val_response_b_encodings = tokenize_function(X_val)
```

</div>
</details>

In [ ]:
# トークナイズ関数
def tokenize_function(df): # 引数としてデータフレームを受け取るトークナイズ関数を定義します
    prompt_encodings = tokenizer( # プロンプトのトークン化
        df['prompt'].tolist(), # 'prompt'列の内容をリストに変換します
        padding='max_length', # 最大長までパディングを行います
        truncation=True, # 長すぎる入力を切り捨てます
        max_length=128, # 最大長を128に設定します
        return_tensors='tf' # TensorFlowテンソル形式で返します
    )
    response_a_encodings = tokenizer( # モデルAの応答のトークン化
        df['response_a'].tolist(), 
        padding='max_length',
        truncation=True,
        max_length=128,
        return_tensors='tf'
    )
    response_b_encodings = tokenizer( # モデルBの応答のトークン化
        df['response_b'].tolist(),
        padding='max_length',
        truncation=True,
        max_length=128,
        return_tensors='tf'
    )
    return prompt_encodings, response_a_encodings, response_b_encodings # トークン化されたプロンプト、応答A、応答Bを返します

# トークナイズを実行し、トレーニングデータとバリデーションデータを処理します
train_prompt_encodings, train_response_a_encodings, train_response_b_encodings = tokenize_function(X_train)
val_prompt_encodings, val_response_a_encodings, val_response_b_encodings = tokenize_function(X_val)

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
import tensorflow as tf
from tensorflow.keras.utils import to_categorical 

# Prepare input features and labels
train_labels = to_categorical(X_train['winner'].tolist(), num_classes=3)
val_labels = to_categorical(X_val['winner'].tolist(), num_classes=3)

# Create TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices((
    {
        'input_ids_prompt': train_prompt_encodings['input_ids'],
        'attention_mask_prompt': train_prompt_encodings['attention_mask'],
        'input_ids_response_a': train_response_a_encodings['input_ids'],
        'attention_mask_response_a': train_response_a_encodings['attention_mask'],
        'input_ids_response_b': train_response_b_encodings['input_ids'],
        'attention_mask_response_b': train_response_b_encodings['attention_mask'],
    },
    train_labels
)).shuffle(1000).batch(1)

val_dataset = tf.data.Dataset.from_tensor_slices((
    {
        'input_ids_prompt': val_prompt_encodings['input_ids'],
        'attention_mask_prompt': val_prompt_encodings['attention_mask'],
        'input_ids_response_a': val_response_a_encodings['input_ids'],
        'attention_mask_response_a': val_response_a_encodings['attention_mask'],
        'input_ids_response_b': val_response_b_encodings['input_ids'],
        'attention_mask_response_b': val_response_b_encodings['attention_mask'],
    },
    val_labels
)).batch(1)

```

</div>
<div class="column-right">

# 日本語訳

```python
import tensorflow as tf # TensorFlowライブラリをインポートします
from tensorflow.keras.utils import to_categorical # カテゴリカルラベルに変換するためのユーティリティをインポートします

# 入力特徴量とラベルを準備します
train_labels = to_categorical(X_train['winner'].tolist(), num_classes=3) # トレーニングラベルをカテゴリカル形式に変換します
val_labels = to_categorical(X_val['winner'].tolist(), num_classes=3) # バリデーションラベルをカテゴリカル形式に変換します

# TensorFlowデータセットを作成します
train_dataset = tf.data.Dataset.from_tensor_slices(( # トレーニングデータセットをテンソルスライスから作成します
    {
        'input_ids_prompt': train_prompt_encodings['input_ids'], # プロンプトの入力ID
        'attention_mask_prompt': train_prompt_encodings['attention_mask'], # プロンプトのアテンションマスク
        'input_ids_response_a': train_response_a_encodings['input_ids'], # 応答Aの入力ID
        'attention_mask_response_a': train_response_a_encodings['attention_mask'], # 応答Aのアテンションマスク
        'input_ids_response_b': train_response_b_encodings['input_ids'], # 応答Bの入力ID
        'attention_mask_response_b': train_response_b_encodings['attention_mask'], # 応答Bのアテンションマスク
    },
    train_labels # 対応するトレーニングラベル
)).shuffle(1000).batch(1) # データをシャッフルし、バッチサイズ1で処理します

val_dataset = tf.data.Dataset.from_tensor_slices(( # バリデーションデータセットも同様に作成します
    {
        'input_ids_prompt': val_prompt_encodings['input_ids'],
        'attention_mask_prompt': val_prompt_encodings['attention_mask'],
        'input_ids_response_a': val_response_a_encodings['input_ids'],
        'attention_mask_response_a': val_response_a_encodings['attention_mask'],
        'input_ids_response_b': val_response_b_encodings['input_ids'],
        'attention_mask_response_b': val_response_b_encodings['attention_mask'],
    },
    val_labels # 対応するバリデーションラベル
)).batch(1) # バッチサイズ1で処理します
```

</div>
</details>

In [ ]:
import tensorflow as tf # TensorFlowライブラリをインポートします
from tensorflow.keras.utils import to_categorical # カテゴリカルラベルに変換するためのユーティリティをインポートします

# 入力特徴量とラベルを準備します
train_labels = to_categorical(X_train['winner'].tolist(), num_classes=3) # トレーニングラベルをカテゴリカル形式に変換します
val_labels = to_categorical(X_val['winner'].tolist(), num_classes=3) # バリデーションラベルをカテゴリカル形式に変換します

# TensorFlowデータセットを作成します
train_dataset = tf.data.Dataset.from_tensor_slices(( # トレーニングデータセットをテンソルスライスから作成します
    {
        'input_ids_prompt': train_prompt_encodings['input_ids'], # プロンプトの入力ID
        'attention_mask_prompt': train_prompt_encodings['attention_mask'], # プロンプトのアテンションマスク
        'input_ids_response_a': train_response_a_encodings['input_ids'], # 応答Aの入力ID
        'attention_mask_response_a': train_response_a_encodings['attention_mask'], # 応答Aのアテンションマスク
        'input_ids_response_b': train_response_b_encodings['input_ids'], # 応答Bの入力ID
        'attention_mask_response_b': train_response_b_encodings['attention_mask'], # 応答Bのアテンションマスク
    },
    train_labels # 対応するトレーニングラベル
)).shuffle(1000).batch(1) # データをシャッフルし、バッチサイズ1で処理します

val_dataset = tf.data.Dataset.from_tensor_slices(( # バリデーションデータセットも同様に作成します
    {
        'input_ids_prompt': val_prompt_encodings['input_ids'],
        'attention_mask_prompt': val_prompt_encodings['attention_mask'],
        'input_ids_response_a': val_response_a_encodings['input_ids'],
        'attention_mask_response_a': val_response_a_encodings['attention_mask'],
        'input_ids_response_b': val_response_b_encodings['input_ids'],
        'attention_mask_response_b': val_response_b_encodings['attention_mask'],
    },
    val_labels # 対応するバリデーションラベル
)).batch(1) # バッチサイズ1で処理します

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python

# Define inputs
input_ids_prompt = tf.keras.layers.Input(shape=(128,), dtype=tf.int32, name="input_ids_prompt")
attention_mask_prompt = tf.keras.layers.Input(shape=(128,), dtype=tf.int32, name="attention_mask_prompt")

input_ids_response_a = tf.keras.layers.Input(shape=(128,), dtype=tf.int32, name="input_ids_response_a")
attention_mask_response_a = tf.keras.layers.Input(shape=(128,), dtype=tf.int32, name="attention_mask_response_a")

input_ids_response_b = tf.keras.layers.Input(shape=(128,), dtype=tf.int32, name="input_ids_response_b")
attention_mask_response_b = tf.keras.layers.Input(shape=(128,), dtype=tf.int32, name="attention_mask_response_b")

```

</div>
<div class="column-right">

# 日本語訳

```python
# 入力を定義します
input_ids_prompt = tf.keras.layers.Input(shape=(128,), dtype=tf.int32, name="input_ids_prompt") # プロンプトの入力IDを受け取るための層を定義します
attention_mask_prompt = tf.keras.layers.Input(shape=(128,), dtype=tf.int32, name="attention_mask_prompt") # プロンプトのアテンションマスクを受け取るための層を定義します

input_ids_response_a = tf.keras.layers.Input(shape=(128,), dtype=tf.int32, name="input_ids_response_a") # 応答Aの入力IDを受け取るための層を定義します
attention_mask_response_a = tf.keras.layers.Input(shape=(128,), dtype=tf.int32, name="attention_mask_response_a") # 応答Aのアテンションマスクを受け取るための層を定義します

input_ids_response_b = tf.keras.layers.Input(shape=(128,), dtype=tf.int32, name="input_ids_response_b") # 応答Bの入力IDを受け取るための層を定義します
attention_mask_response_b = tf.keras.layers.Input(shape=(128,), dtype=tf.int32, name="attention_mask_response_b") # 応答Bのアテンションマスクを受け取るための層を定義します
```

</div>
</details>

In [ ]:
# 入力を定義します
input_ids_prompt = tf.keras.layers.Input(shape=(128,), dtype=tf.int32, name="input_ids_prompt") # プロンプトの入力IDを受け取るための層を定義します
attention_mask_prompt = tf.keras.layers.Input(shape=(128,), dtype=tf.int32, name="attention_mask_prompt") # プロンプトのアテンションマスクを受け取るための層を定義します

input_ids_response_a = tf.keras.layers.Input(shape=(128,), dtype=tf.int32, name="input_ids_response_a") # 応答Aの入力IDを受け取るための層を定義します
attention_mask_response_a = tf.keras.layers.Input(shape=(128,), dtype=tf.int32, name="attention_mask_response_a") # 応答Aのアテンションマスクを受け取るための層を定義します

input_ids_response_b = tf.keras.layers.Input(shape=(128,), dtype=tf.int32, name="input_ids_response_b") # 応答Bの入力IDを受け取るための層を定義します
attention_mask_response_b = tf.keras.layers.Input(shape=(128,), dtype=tf.int32, name="attention_mask_response_b") # 応答Bのアテンションマスクを受け取るための層を定義します

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
import tensorflow as tf
from transformers import TFBertModel

class BertEmbeddingLayer(tf.keras.layers.Layer):
    def __init__(self, bert_model_name='bert-base-uncased', **kwargs):
        super(BertEmbeddingLayer, self).__init__(**kwargs)
        self.bert = bert_model
        
    def call(self, inputs):
        input_ids, attention_mask = inputs
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        return outputs.last_hidden_state[:, 0, :]  # CLS token embedding

# Initialize the custom BERT layer
bert_layer = BertEmbeddingLayer()

```

</div>
<div class="column-right">

# 日本語訳

```python
import tensorflow as tf # TensorFlowライブラリをインポートします
from transformers import TFBertModel # TensorFlow用BERTモデルをインポートします

class BertEmbeddingLayer(tf.keras.layers.Layer): # カスタムBERT埋め込み層の定義
    def __init__(self, bert_model_name='bert-base-uncased', **kwargs): # コンストラクタ
        super(BertEmbeddingLayer, self).__init__(**kwargs) # 親クラスのコンストラクタを呼び出します
        self.bert = bert_model # BERTモデルを初期化します
        
    def call(self, inputs): # 層が呼び出されたときの処理を定義します
        input_ids, attention_mask = inputs # 入力の分解
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask) # BERTモデルに入力を渡して出力を取得します
        return outputs.last_hidden_state[:, 0, :]  # CLSトークンの埋め込みを返します（最初のトークンに対応）

# カスタムBERT層を初期化します
bert_layer = BertEmbeddingLayer()
```

</div>
</details>

In [ ]:
import tensorflow as tf # TensorFlowライブラリをインポートします
from transformers import TFBertModel # TensorFlow用BERTモデルをインポートします

class BertEmbeddingLayer(tf.keras.layers.Layer): # カスタムBERT埋め込み層の定義
    def __init__(self, bert_model_name='bert-base-uncased', **kwargs): # コンストラクタ
        super(BertEmbeddingLayer, self).__init__(**kwargs) # 親クラスのコンストラクタを呼び出します
        self.bert = bert_model # BERTモデルを初期化します
        
    def call(self, inputs): # 層が呼び出されたときの処理を定義します
        input_ids, attention_mask = inputs # 入力の分解
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask) # BERTモデルに入力を渡して出力を取得します
        return outputs.last_hidden_state[:, 0, :]  # CLSトークンの埋め込みを返します（最初のトークンに対応）

# カスタムBERT層を初期化します
bert_layer = BertEmbeddingLayer()

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python

prompt_embeddings = bert_layer([input_ids_prompt, attention_mask_prompt])
response_a_embeddings = bert_layer([input_ids_response_a, attention_mask_response_a])
response_b_embeddings = bert_layer([input_ids_response_b, attention_mask_response_b])

# Concatenate embeddings
combined_embeddings = tf.keras.layers.Concatenate()([prompt_embeddings, response_a_embeddings, response_b_embeddings])

```

</div>
<div class="column-right">

# 日本語訳

```python
prompt_embeddings = bert_layer([input_ids_prompt, attention_mask_prompt]) # プロンプトの埋め込みを取得します
response_a_embeddings = bert_layer([input_ids_response_a, attention_mask_response_a]) # 応答Aの埋め込みを取得します
response_b_embeddings = bert_layer([input_ids_response_b, attention_mask_response_b]) # 応答Bの埋め込みを取得します

# 埋め込みを結合します
combined_embeddings = tf.keras.layers.Concatenate()([prompt_embeddings, response_a_embeddings, response_b_embeddings]) # プロンプト、応答A、応答Bの埋め込みを結合します
```

</div>
</details>

In [ ]:
prompt_embeddings = bert_layer([input_ids_prompt, attention_mask_prompt]) # プロンプトの埋め込みを取得します
response_a_embeddings = bert_layer([input_ids_response_a, attention_mask_response_a]) # 応答Aの埋め込みを取得します
response_b_embeddings = bert_layer([input_ids_response_b, attention_mask_response_b]) # 応答Bの埋め込みを取得します

# 埋め込みを結合します
combined_embeddings = tf.keras.layers.Concatenate()([prompt_embeddings, response_a_embeddings, response_b_embeddings]) # プロンプト、応答A、応答Bの埋め込みを結合します

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python

dense_layer = tf.keras.layers.Dense(256, activation='relu')(combined_embeddings)
dropout_layer = tf.keras.layers.Dropout(0.2)(dense_layer)
output_layer = tf.keras.layers.Dense(3, activation='softmax')(dropout_layer)

# Build and compile the model
model = tf.keras.Model(inputs=[
    input_ids_prompt, attention_mask_prompt,
    input_ids_response_a, attention_mask_response_a,
    input_ids_response_b, attention_mask_response_b
], outputs=output_layer)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5), loss='categorical_crossentropy', metrics=['accuracy'])

```

</div>
<div class="column-right">

# 日本語訳

```python
dense_layer = tf.keras.layers.Dense(256, activation='relu')(combined_embeddings) # 結合された埋め込みを入力として、256ユニットの全結合層を定義します
dropout_layer = tf.keras.layers.Dropout(0.2)(dense_layer) # ドロップアウト層を追加して過学習を防ぎます（ドロップアウト率は20%）
output_layer = tf.keras.layers.Dense(3, activation='softmax')(dropout_layer) # 出力層を定義します。クラス数は3（勝者モデルA、モデルB、引き分け）

# モデルを構築し、コンパイルします
model = tf.keras.Model(inputs=[ # モデルの入力を指定します
    input_ids_prompt, attention_mask_prompt,
    input_ids_response_a, attention_mask_response_a,
    input_ids_response_b, attention_mask_response_b
], outputs=output_layer) # モデルの出力を指定します

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5), loss='categorical_crossentropy', metrics=['accuracy']) # モデルをコンパイルします。オプティマイザーはAdam、損失関数はカテゴリカルクロスエントロピー、評価指標は精度を使用します
```

</div>
</details>

In [ ]:
dense_layer = tf.keras.layers.Dense(256, activation='relu')(combined_embeddings) # 結合された埋め込みを入力として、256ユニットの全結合層を定義します
dropout_layer = tf.keras.layers.Dropout(0.2)(dense_layer) # ドロップアウト層を追加して過学習を防ぎます（ドロップアウト率は20%）
output_layer = tf.keras.layers.Dense(3, activation='softmax')(dropout_layer) # 出力層を定義します。クラス数は3（勝者モデルA、モデルB、引き分け）

# モデルを構築し、コンパイルします
model = tf.keras.Model(inputs=[ # モデルの入力を指定します
    input_ids_prompt, attention_mask_prompt,
    input_ids_response_a, attention_mask_response_a,
    input_ids_response_b, attention_mask_response_b
], outputs=output_layer) # モデルの出力を指定します

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5), loss='categorical_crossentropy', metrics=['accuracy']) # モデルをコンパイルします。オプティマイザーはAdam、損失関数はカテゴリカルクロスエントロピー、評価指標は精度を使用します

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python

history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=1
)

```

</div>
<div class="column-right">

# 日本語訳

```python
history = model.fit( # モデルのトレーニングを開始します
    train_dataset, # トレーニングデータセットを指定します
    validation_data=val_dataset, # バリデーションデータセットを指定します
    epochs=1 # エポック数は1に設定します（1回のトレーニングで全データを1周します）
)
```

</div>
</details>

In [ ]:
history = model.fit( # モデルのトレーニングを開始します
    train_dataset, # トレーニングデータセットを指定します
    validation_data=val_dataset, # バリデーションデータセットを指定します
    epochs=1 # エポック数は1に設定します（1回のトレーニングで全データを1周します）
)

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
def encode_text(texts, max_length):
    return tokenizer(
        texts,
        truncation=True,
        padding='max_length',
        max_length=max_length,
        return_tensors='tf'
    )
```

</div>
<div class="column-right">

# 日本語訳

```python
def encode_text(texts, max_length): # テキストをエンコードする関数を定義します
    return tokenizer( # トークナイザーを使用してテキストをトークン化します
        texts,
        truncation=True, # 長すぎるテキストを切り捨てます
        padding='max_length', # 最大長までパディングします
        max_length=max_length, # 最大長を指定します
        return_tensors='tf' # TensorFlowテンソル形式で返します
    )
```

</div>
</details>

In [ ]:
def encode_text(texts, max_length): # テキストをエンコードする関数を定義します
    return tokenizer( # トークナイザーを使用してテキストをトークン化します
        texts,
        truncation=True, # 長すぎるテキストを切り捨てます
        padding='max_length', # 最大長までパディングします
        max_length=max_length, # 最大長を指定します
        return_tensors='tf' # TensorFlowテンソル形式で返します
    )

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
test
```

</div>
<div class="column-right">

# 日本語訳

```python
test # テストデータセットを表示します。これによりテストデータの内容を確認できます。
```

</div>
</details>

In [ ]:
test # テストデータセットを表示します。これによりテストデータの内容を確認できます。

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
max_length = 128  # Adjust according to your model's max sequence length
input_ids_prompt = encode_text(test['prompt'].tolist(), max_length)
input_ids_response_a = encode_text(test['response_a'].tolist(), max_length)
input_ids_response_b = encode_text(test['response_b'].tolist(), max_length)
```

</div>
<div class="column-right">

# 日本語訳

```python
max_length = 128  # モデルの最大シーケンス長に応じて調整します
input_ids_prompt = encode_text(test['prompt'].tolist(), max_length) # テストセットのプロンプトをエンコードします
input_ids_response_a = encode_text(test['response_a'].tolist(), max_length) # テストセットの応答Aをエンコードします
input_ids_response_b = encode_text(test['response_b'].tolist(), max_length) # テストセットの応答Bをエンコードします
```

</div>
</details>

In [ ]:
max_length = 128  # モデルの最大シーケンス長に応じて調整します
input_ids_prompt = encode_text(test['prompt'].tolist(), max_length) # テストセットのプロンプトをエンコードします
input_ids_response_a = encode_text(test['response_a'].tolist(), max_length) # テストセットの応答Aをエンコードします
input_ids_response_b = encode_text(test['response_b'].tolist(), max_length) # テストセットの応答Bをエンコードします

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
predictions = model.predict({
    'input_ids_prompt': input_ids_prompt['input_ids'],
    'attention_mask_prompt': input_ids_prompt['attention_mask'],
    'input_ids_response_a': input_ids_response_a['input_ids'],
    'attention_mask_response_a': input_ids_response_a['attention_mask'],
    'input_ids_response_b': input_ids_response_b['input_ids'],
    'attention_mask_response_b': input_ids_response_b['attention_mask']
})
```

</div>
<div class="column-right">

# 日本語訳

```python
predictions = model.predict({ # モデルを使って予測を行います
    'input_ids_prompt': input_ids_prompt['input_ids'], # プロンプトの入力ID
    'attention_mask_prompt': input_ids_prompt['attention_mask'], # プロンプトのアテンションマスク
    'input_ids_response_a': input_ids_response_a['input_ids'], # 応答Aの入力ID
    'attention_mask_response_a': input_ids_response_a['attention_mask'], # 応答Aのアテンションマスク
    'input_ids_response_b': input_ids_response_b['input_ids'], # 応答Bの入力ID
    'attention_mask_response_b': input_ids_response_b['attention_mask'] # 応答Bのアテンションマスク
}) # これにより、テストデータに対してモデルが予測を行います
```

</div>
</details>

In [ ]:
predictions = model.predict({ # モデルを使って予測を行います
    'input_ids_prompt': input_ids_prompt['input_ids'], # プロンプトの入力ID
    'attention_mask_prompt': input_ids_prompt['attention_mask'], # プロンプトのアテンションマスク
    'input_ids_response_a': input_ids_response_a['input_ids'], # 応答Aの入力ID
    'attention_mask_response_a': input_ids_response_a['attention_mask'], # 応答Aのアテンションマスク
    'input_ids_response_b': input_ids_response_b['input_ids'], # 応答Bの入力ID
    'attention_mask_response_b': input_ids_response_b['attention_mask'] # 応答Bのアテンションマスク
}) # これにより、テストデータに対してモデルが予測を行います

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
print("done till here")
```

</div>
<div class="column-right">

# 日本語訳

```python
print("done till here") # ここまでの処理が完了したことを表示します
```

</div>
</details>

In [ ]:
print("done till here") # ここまでの処理が完了したことを表示します

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
results=pd.DataFrame({
    'ID':test['id'],
    'winner_model_a':predictions[0],
    'winner_model_b':predictions[1],
    'winner_tie':predictions[2],
    
})
```

</div>
<div class="column-right">

# 日本語訳

```python
results = pd.DataFrame({ # 予測結果をデータフレームにまとめます
    'ID': test['id'], # テストセットのID列
    'winner_model_a': predictions[0], # モデルAの勝者確率
    'winner_model_b': predictions[1], # モデルBの勝者確率
    'winner_tie': predictions[2], # 引き分けの確率
})
```

</div>
</details>

In [ ]:
results = pd.DataFrame({ # 予測結果をデータフレームにまとめます
    'ID': test['id'], # テストセットのID列
    'winner_model_a': predictions[0], # モデルAの勝者確率
    'winner_model_b': predictions[1], # モデルBの勝者確率
    'winner_tie': predictions[2], # 引き分けの確率
})

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
results.to_csv('/kaggle/working/submission.csv', index=False)
```

</div>
<div class="column-right">

# 日本語訳

```python
results.to_csv('/kaggle/working/submission.csv', index=False) # 予測結果をCSVファイルとして保存します
# ファイル名は'submission.csv'で、インデックスは保存しません
```

</div>
</details>

In [ ]:
results.to_csv('/kaggle/working/submission.csv', index=False) # 予測結果をCSVファイルとして保存します
# ファイル名は'submission.csv'で、インデックスは保存しません

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python

```

</div>
<div class="column-right">

# 日本語訳

```python

```

</div>
</details>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python

```

</div>
<div class="column-right">

# 日本語訳

```python

```

</div>
</details>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python

```

</div>
<div class="column-right">

# 日本語訳

```python

```

</div>
</details>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python

```

</div>
<div class="column-right">

# 日本語訳

```python

```

</div>
</details>